
# 🧠 Multi-Agent Debate DAG (LangGraph + LLMs)

This notebook demonstrates a fully autonomous, multi-round debate simulation between two AI agents (Scientist and Philosopher) using Groq-hosted LLaMA-3 models and LangGraph.

**Key Features:**
- 8-round debate between two agents
- Structured memory of arguments
- Automated judging by a 70B model
- CLI + Notebook compatibility

**⚠️ Replace `GROQ_API_KEY` with your actual key**


In [ ]:

!pip install langgraph langchain langchain-core langchain-groq rich


In [ ]:

import os
os.environ["GROQ_API_KEY"] = "sk-your-key-here"  # 🔁 Replace with your own key


In [ ]:

from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from typing import List, Tuple, Any

class DebateAgent:
    def __init__(self, name: str, model: str = "llama3-8b-8192"):
        self.name = name
        self.llm = ChatGroq(model_name=model, temperature=0.7)
        self.prompt = ChatPromptTemplate.from_messages([
            ("system", "You are {name}, an expert in debate. Debate the topic: {topic}."),
            ("human", "{history}")
        ])

    def respond(self, topic: str, history: List[Tuple[str, str]]) -> str:
        transcript = "\n".join([f"{speaker}: {arg}" for speaker, arg in history])
        chain = self.prompt.partial(name=self.name) | self.llm
        return chain.invoke({"topic": topic, "history": transcript}).content.strip()


In [ ]:

class DebateJudge:
    def __init__(self, model: str = "llama3-70b-8192"):
        self.llm = ChatGroq(model_name=model, temperature=0.2)
        self.prompt = ChatPromptTemplate.from_messages([
            ("system", '''You are an expert debate judge. Provide:
1. A comprehensive summary
2. The winner ("Scientist" or "Philosopher")
3. Justification for your decision'''),
            ("human", "Debate Topic: {topic}\nDebate Transcript:\n{transcript}")
        ])

    def judge(self, topic: str, history: List[Tuple[str, str]]) -> dict:
        transcript = "\n".join([f"{s}: {a}" for s, a in history])
        result = (self.prompt | self.llm).invoke({"topic": topic, "transcript": transcript}).content
        winner = "Scientist" if "scientist" in result.lower() else "Philosopher" if "philosopher" in result.lower() else "Draw"
        return {"summary": result, "winner": winner}


In [ ]:

from langgraph.graph import StateGraph

class DebateMemory:
    def __init__(self):
        self.topic = ""
        self.history = []

    def set_topic(self, topic: str):
        self.topic = topic

    def add_argument(self, speaker: str, argument: str):
        self.history.append((speaker, argument))

    def get_history(self):
        return self.history

    def get(self, _):
        return self.topic


In [ ]:

from typing import Dict

class UserInputNode:
    def __call__(self, state: Dict) -> Dict:
        topic = input("Enter topic: ")
        state["memory"].set_topic(topic)
        return {"topic": topic, "memory": state["memory"], "round": 0, "next_speaker": "Scientist"}

class AgentNode:
    def __init__(self, name: str, agent):
        self.name = name
        self.agent = agent

    def __call__(self, state: Dict) -> Dict:
        topic = state["memory"].get("topic")
        history = state["memory"].get_history()
        response = self.agent.respond(topic, history)
        state["memory"].add_argument(self.name, response)
        return {"memory": state["memory"], "last_speaker": self.name, "last_argument": response}

class RoundControlNode:
    def __init__(self, max_rounds: int):
        self.max_rounds = max_rounds

    def __call__(self, state: Dict) -> Dict:
        current_round = state.get("round", 0) + 1
        done = current_round >= self.max_rounds
        next_speaker = "Philosopher" if state.get("last_speaker") == "Scientist" else "Scientist"
        return {"round": current_round, "done": done, "next_speaker": next_speaker, "memory": state["memory"]}

class JudgeNode:
    def __init__(self, judge):
        self.judge = judge

    def __call__(self, state: Dict) -> Dict:
        topic = state["memory"].get("topic")
        history = state["memory"].get_history()
        result = self.judge.judge(topic, history)
        print("📢 Judge Verdict:", result)
        return {"judgment": result}


In [ ]:

def build_graph(agent_a, agent_b, judge):
    g = StateGraph(state_schema=dict)
    g.add_node("user_input", UserInputNode())
    g.add_node("agent_a", AgentNode("Scientist", agent_a))
    g.add_node("agent_b", AgentNode("Philosopher", agent_b))
    g.add_node("round_control", RoundControlNode(8))
    g.add_node("judge", JudgeNode(judge))
    
    g.set_entry_point("user_input")
    g.add_edge("user_input", "round_control")
    g.add_conditional_edges("round_control", lambda s: "judge" if s.get("done") else ("agent_a" if s["next_speaker"] == "Scientist" else "agent_b"), {
        "agent_a": "agent_a",
        "agent_b": "agent_b",
        "judge": "judge"
    })
    g.add_edge("agent_a", "round_control")
    g.add_edge("agent_b", "round_control")
    g.set_finish_point("judge")
    return g.compile()


In [ ]:

memory = DebateMemory()
agent_a = DebateAgent("Scientist")
agent_b = DebateAgent("Philosopher")
judge = DebateJudge()

app = build_graph(agent_a, agent_b, judge)

state = {"memory": memory, "input": {"prompt": "Enter topic:", "input_func": input}}
for output in app.stream(state):
    print(output)
